# SBTi-Finance Tool - Calculate portfolio coverage
This notebook provides a simple way to calculate portfolio coverage with a cut-off date specified by the user. The intention is to allow the SBTi TVT as well as financial institutions to calculate the portfolio coverage at a date in the past corresponding to a base date or base year for the FI's portfolio target.

This notebook currently only supports aggregation via the WATS method.

This notebook does not calculate any temperature scores.

### Install the necessary Python modules
This is only required if you have not already installed the module.

In [ ]:
# Ensure required packages are installed
import pandas as pd
import openpyxl
import requests
from datetime import datetime
import re
from difflib import SequenceMatcher

## Create the data directory and download the example portfolio
We have prepared dummy data for you to be able to run the tool as it is to familiarise yourself with how it works. To use your own data; please check out to the [Data Requirements section](https://sciencebasedtargets.github.io/SBTi-finance-tool/DataRequirements.html) of the technical documentation for more details on data requirements and formatting. 

*The dummy data may include some company names, but the data associated with those company names is completely random and any similarities with real world data is purely coincidental.

In [ ]:
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_portfolio.csv", "data/example_portfolio.csv")

## Load your portfolio
The example portfolio is stored as a .csv file. Alternatively, you can upload an .xlsx file, just choose one of the loading options below.

You can upload your portfolio file using the folder icon on the left hand side of the screen. Once you have uploaded your file, you can load it into the notebook using the code below.

The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column. To calculate the weighted portfolio coverage the data also needs to include identifers for the portfolios constituents, preferably LEI, but ISIN and exact matching entity names are recognized by the SBTi data.
The following column names headers are required to upload the file to the tool, please note the the tool will not run unless these exact headers are included:

company_name: Name of the company in your portfolio - each row must have a unique name  
company_id : Unique identifier for the company in your portfolio  
isin : The ISIN of the company in your portfolio, used to identify the company e.g. for SBTi status *  
lei : Legal Entity Identifier for the company in your portfolio, used identify the company e.g. for SBTi status *  
investment_value: Needed to weight the portfolio coverage by the value of the investment in the company  
target: Not needed for the tool to run but is included to be compatible with the finance tool portfolio format, used to reference if any company with LEI, ISIN, or exact matching name from the SBTi Companies Taking Action table

\* These rows may be left blank but are needed to check the SBTi status of the company. If you do not have this data, please leave the rows blank and the tool will still run, but the SBTi status will not be included in the output.

Please see the technical documentation on [Data Legends](https://sciencebasedtargets.github.io/SBTi-finance-tool/Legends.html#) for details on data requirements. 

### Load the portfolio from a CSV file or an xlsx file
Enter the path to your portfolio file inside the quotation marks below. Then remove the # at the beginning of the appropriate line and run the cell.

In [ ]:
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")
#df_portfolio = pd.read_excel("data/example_portfolio.xlsx", engine="openpyxl") # .xlsx format

#Use your local file instead
#my_file_path = "path/to/your/portfolio_file.csv"
#my_file_path = "//SBTi-finance-tool/examples/data/example_portfolio.csv"
#df_portfolio = pd.read_csv(my_file_path, encoding="utf-8")

In [ ]:
# Add this code right after loading the portfolio
# Convert all column names to snake_case format
def convert_to_snake_case(name):
    """Convert any string to snake_case format"""
    import re
    # Handle CamelCase and PascalCase
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    # Convert to lowercase
    s3 = s2.lower()
    # Replace spaces and other separators with underscores
    s4 = re.sub(r'[^a-z0-9_]', '_', s3)
    # Remove duplicate underscores
    s5 = re.sub(r'_+', '_', s4)
    # Remove leading and trailing underscores
    return s5.strip('_')

# Apply conversion to all column names
original_columns = df_portfolio.columns.tolist()
df_portfolio.columns = [convert_to_snake_case(col) for col in df_portfolio.columns]

In [ ]:
# Detect column format and standardize for flexibility
# Support both new format (isin, lei) and old format (company_isin, company_lei)

if 'isin' in df_portfolio.columns and 'company_isin' in df_portfolio.columns:
    print("Warning: Both 'isin' and 'company_isin' columns found. Using 'company_isin'.")
    portfolio_isin_col = 'company_isin'
elif 'company_isin' in df_portfolio.columns:
    print("Using old format: 'company_isin' column detected.")
    portfolio_isin_col = 'company_isin'
elif 'isin' in df_portfolio.columns:
    print("Using new format: 'isin' column detected.")
    portfolio_isin_col = 'isin'
else:
    print("Warning: No ISIN column found.")
    portfolio_isin_col = None

if 'lei' in df_portfolio.columns and 'company_lei' in df_portfolio.columns:
    print("Warning: Both 'lei' and 'company_lei' columns found. Using 'company_lei'.")
    portfolio_lei_col = 'company_lei'
elif 'company_lei' in df_portfolio.columns:
    print("Using old format: 'company_lei' column detected.")
    portfolio_lei_col = 'company_lei'
elif 'lei' in df_portfolio.columns:
    print("Using new format: 'lei' column detected.")
    portfolio_lei_col = 'lei'
else:
    print("Warning: No LEI column found.")
    portfolio_lei_col = None

print(f"Portfolio format detected - ISIN column: {portfolio_isin_col}, LEI column: {portfolio_lei_col}")

In [ ]:
# Change the column names to match the API if the snakecase conversion did not work
#df_portfolio.rename(columns={'Company Name': 'company_name', 'ISIN': 'isin'}, inplace=True)

# Check for duplicate values in the 'company_id' column
duplicate_ids = df_portfolio[df_portfolio.duplicated('company_id', keep=False)]

if not duplicate_ids.empty:
    print("Error: Duplicate values found in the 'company_id' column:")
    print(duplicate_ids)
else:
    print("No duplicate values found in the 'company_id' column.")

## Enter the date to be used in calculating the portfolio coverage.
The date has to be older than today's date.
The format is: 

In [ ]:
year = 2020 #enter the year for which you want to calculate the portfolio coverage
month = 12 #enter the month for which you want to calculate the portfolio coverage
day = 31 #enter the day for which you want to calculate the portfolio coverage

In [ ]:
user_date = datetime(year, month, day)

Now load the CTA file (Companies Taking Action) from the SBTi website.

In [ ]:
# Use the enhanced SBTi class for consistent format handling
from SBTi.data.sbti import SBTi

print("Loading SBTi Companies Taking Action data...")
sbti_provider = SBTi()
print(f"SBTi format: {getattr(sbti_provider, 'format_type', 'detected automatically')}")
print(f"Companies loaded: {len(sbti_provider.targets)}")

# Access the processed targets data (already filtered and formatted)
cta_file = sbti_provider.targets.copy()

In [ ]:
cta_file.head()

## Filter the CTA file
Filter the CTA file to create a dataframe that has one row per company with the columns "action" and "target".
If Action = Target then only keep the rows where Target = Near-term.
        

In [ ]:
# The SBTi class has already processed and filtered the data
# Just work with the pre-processed targets
targets = cta_file.copy()

# Filter for companies that have "Target" in the action field  
companies_with_targets = targets[targets[sbti_provider.c.COL_ACTION] == sbti_provider.c.VALUE_ACTION_TARGET]

print(f"Total companies with targets: {len(companies_with_targets)}")

# Get unique company names with targets
unique_companies_with_targets = companies_with_targets[sbti_provider.c.COL_COMPANY_NAME].unique()
total_companies_with_targets = len(unique_companies_with_targets)

# Total unique companies in SBTi database
total_companies_in_sbti = len(targets[sbti_provider.c.COL_COMPANY_NAME].unique())

# Create a new dataframe with one row per company (to avoid duplicates in counting)
unique_companies_df = companies_with_targets.drop_duplicates(subset=[sbti_provider.c.COL_COMPANY_NAME])

# Create sets for companies with different identifiers
companies_with_isin = set(unique_companies_df[unique_companies_df[sbti_provider.c.COL_COMPANY_ISIN].notna()][sbti_provider.c.COL_COMPANY_NAME])
companies_with_lei = set(unique_companies_df[unique_companies_df[sbti_provider.c.COL_COMPANY_LEI].notna()][sbti_provider.c.COL_COMPANY_NAME])

# Get unique ISINs and LEIs
all_isin_set = set(companies_with_targets[sbti_provider.c.COL_COMPANY_ISIN].dropna())
all_lei_set = set(companies_with_targets[sbti_provider.c.COL_COMPANY_LEI].dropna())

# Calculate the different categories
companies_with_both = companies_with_isin.intersection(companies_with_lei)
companies_with_only_isin = companies_with_isin - companies_with_both
companies_with_only_lei = companies_with_lei - companies_with_both
companies_with_neither = set(unique_companies_with_targets) - companies_with_isin - companies_with_lei

# Count companies in each category
total_companies_with_both = len(companies_with_both)
total_companies_with_only_isin = len(companies_with_only_isin)
total_companies_with_only_lei = len(companies_with_only_lei)
total_companies_without_identifiers = len(companies_with_neither)

# Print the analysis
print(f"Total unique companies in the SBTi database: {total_companies_in_sbti}")
print(f"Total companies with targets in SBTi database: {total_companies_with_targets}")
print(f"Total unique ISINs with targets: {len(all_isin_set)}")
print(f"Total unique LEIs with targets: {len(all_lei_set)}")
print(f"Companies with targets with both ISIN and LEI: {total_companies_with_both}")
print(f"Companies with targets with only ISIN (no LEI): {total_companies_with_only_isin}")
print(f"Companies with targets with only LEI (no ISIN): {total_companies_with_only_lei}")
print(f"Companies with targets but no LEI or ISIN: {total_companies_without_identifiers}")

# Verification
calculated_total = (total_companies_with_both + 
                   total_companies_with_only_isin + 
                   total_companies_with_only_lei + 
                   total_companies_without_identifiers)
print(f"Sum of all categories: {calculated_total}")
print(f"Matches total companies with targets: {calculated_total == total_companies_with_targets}")

## Filter out dates
Now filter out rows according to the provided date

In [ ]:
# Convert the "Date Published" column to datetime type
df_targets = companies_with_targets.copy()  
df_targets[sbti_provider.c.COL_DATE_PUBLISHED] = pd.to_datetime(df_targets[sbti_provider.c.COL_DATE_PUBLISHED])

# Filter rows based on user-entered date
filtered_df = df_targets.loc[df_targets[sbti_provider.c.COL_DATE_PUBLISHED] <= user_date]
filtered_df = filtered_df[filtered_df[sbti_provider.c.COL_COMPANY_ISIN].notnull() | filtered_df[sbti_provider.c.COL_COMPANY_LEI].notnull()]  

# Create a set of company names from the filtered SBTi data
filtered_df['company_name_lower'] = filtered_df[sbti_provider.c.COL_COMPANY_NAME].str.lower()
company_name_set = set(filtered_df['company_name_lower'].dropna())

## Check CTA file for companies with validated targets

In [ ]:
# Create sets for matching
isin_set = set(filtered_df[sbti_provider.c.COL_COMPANY_ISIN])
lei_set = set(filtered_df[sbti_provider.c.COL_COMPANY_LEI])

# Flexible validation function that works with both old and new column formats
def is_validated(row):
    # Check LEI (use detected column name)
    if portfolio_lei_col and pd.notna(row.get(portfolio_lei_col)) and row.get(portfolio_lei_col) in all_lei_set:
        return True
    
    # Check ISIN (use detected column name)
    if portfolio_isin_col and pd.notna(row.get(portfolio_isin_col)) and row.get(portfolio_isin_col) in all_isin_set:
        return True
    
    # Check company name
    if pd.notna(row.get('company_name')):
        company_name_lower = row.get('company_name').lower()
        if company_name_lower in company_name_set:
            return True
    
    # If none of the conditions are met
    return False

# Apply the function to create the 'validated' column
df_portfolio['validated'] = df_portfolio.apply(is_validated, axis=1)

print(f"Validation completed using ISIN column: {portfolio_isin_col}, LEI column: {portfolio_lei_col}")

## Portfolio coverage

The portfolio coverage provides insights in the proportion of the portfolio that has set SBTi-approved GHG emissions reduction targets. Only companies with SBTi-status "Approved" are included in the portfolio coverage.

To calculate the portfolio coverage we use the same aggregation methods we use for the Portfolio Temperature Score. Currently, in this simplified notebook, only the "Weighted Average Temperature Score" (WATS) is used. For more details on aggregation methods and the portfolio coverage method, please refer to the [methodology document](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf) sections 3.2 and also turn to notebook 4 (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/4_portfolio_aggregations.ipynb) or [GitHub](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/4_portfolio_aggregations.ipynb)) for more aggregation examples.

In [ ]:
total_investment_weight = df_portfolio['investment_value'].sum()
try:
    pc_weighted = df_portfolio.apply(
        lambda row: (row['investment_value'] * row['validated'])
        / total_investment_weight,
        axis=1,
    )
except ZeroDivisionError:
    raise ValueError("The portfolio weight is not allowed to be zero")
pc_result = round(pc_weighted.sum(), 2)
print(f"Portfolio coverage is: {pc_result*100}%.")

In [ ]:
#Print the Total and Validated Investment Values
total_investment_weight = df_portfolio["investment_value"].sum()
validated_investment_sum = df_portfolio.loc[df_portfolio["validated"] == True, "investment_value"].sum()

print(f"Total Investment Value: {total_investment_weight:,.2f}")
print(f"Validated Investment Value: {validated_investment_sum:,.2f}")
print(f"Percentage of Portfolio Value with Validated Targets: {(validated_investment_sum/total_investment_weight)*100:.2f}%")
print(f"Total Companies in Portfolio: {len(df_portfolio)}")
print(f"Validated Companies: {df_portfolio['validated'].sum()}")
print(f"Percentage of Companies with Validated Targets: {(df_portfolio['validated'].sum()/len(df_portfolio))*100:.2f}%")

#Show the first few validated companies
print("\nSample of companies with validated targets:")
print(df_portfolio[df_portfolio["validated"] == True][["company_name", "investment_value", "validated"]].head(10))

Updated counting function to test

In [ ]:
distinct_company_count = df_portfolio['company_name'].nunique()
validated_companies = df_portfolio[df_portfolio['validated']]['company_name'].nunique()

print(f"Total Distinct Companies: {distinct_company_count}")
print(f"Distinct Validated Companies: {validated_companies}")
print(f"Percentage of Distinct Companies with Validated Targets: {(validated_companies/distinct_company_count)*100:.2f}%")

# Original counting method - counts rows, not necessarily distinct companies
print(f"Total Portfolio Rows: {len(df_portfolio)}")
print(f"Validated Rows: {df_portfolio['validated'].sum()}")
print(f"Percentage of Rows with Validated Targets: {(df_portfolio['validated'].sum()/len(df_portfolio))*100:.2f}%")

#Show the first few validated companies
print("\nSample of companies with validated targets:")
print(df_portfolio[df_portfolio["validated"] == True][["company_name", "investment_value", "validated"]].head(10))

## Save the portfolio
If you want to save the portfolio, you can use the following code in the following cell. 

In [ ]:
df_portfolio.to_csv('data/validated_portfolio.csv', index=False)